# Calculate cells movement

### calculating 3D cell movements and saving in csv file.

### 3次元的なROIの移動距離を算出するコードです。


In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
import numpy as np


from __future__ import annotations

import re
from glob import glob
from pathlib import Path

import numpy as np

from scipy import ndimage


from CellTracker.stardistwrapper import load_2d_slices_at_time

In [2]:
def calculate_center(tracked_labels_path :str, raw_path: str | dict):
    tracked_labels_path=Path(tracked_labels_path)
    filenames = glob(str(tracked_labels_path / "*t*.tif"))
    assert len(filenames) > 0, f"No labels files were found in {tracked_labels_path / '*t*.tif'}"
    numbers = [int(re.findall(r"t(\d+)", f)[0]) for f in filenames]
    smallest_number = min(numbers)
    largest_number = max(numbers)
    for t in range(smallest_number, largest_number + 1):
        print(f"t={t}...", end="\r")
        try:
            # Load 2D slices at time t
            raw = load_2d_slices_at_time(raw_path, t=t)
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Raw images at t={t - 1} cannot be loaded! Stop calculation!")
            break

        try:
            # Load 2D slices at time t
            labels_img = load_2d_slices_at_time(str(tracked_labels_path / "*t%06i*.tif"), t=t, do_normalize=False)
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Label images at t={t - 1} cannot be loaded!")
            if t == smallest_number:
                print("Warning: stop calculation!")
                break
            else:
                print(f"Warning: skip volume {t - 1}!")
                cell_position[t - smallest_number, :] = np.nan
                continue

        if t == smallest_number:
            cell_num = np.max(labels_img)
            label_number = list(range(cell_num+1))
            cell_position = np.zeros((largest_number - smallest_number + 1,cell_num, 3))


    


        center_of_labels = ndimage.center_of_mass(raw, labels_img,label_number[1:cell_num+1])
        cell_position[t-smallest_number,:] = center_of_labels[:]
        


    return cell_position

In [3]:
raw_path = {"h5_file":r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=566-665\raw_t=566~665.h5",
            "dset": "default",
            "channel": 0}


tracked_labels_path =r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=566-665\merged_5d_566-665"

#raw_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_reconst\*t%04d*.tif"


center_of_labels = calculate_center(tracked_labels_path = tracked_labels_path, raw_path=raw_path)



In [5]:
np.save(r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=566~665\center_of_labels_t=566~665.npy" ,center_of_labels )
# center_of_labels=np.load("center_of_labels_t=201-221.npy")
print(center_of_labels.shape)

(100, 2, 3)


In [ ]:
center_of_labels=np.load(r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=226~364\center_of_labels_t=226~364.npy" )

## 線虫動画用

In [ ]:
from scipy.spatial import distance

csv_filename=r"P:\Tardigrade\Tardigrada_mukumoto\ana\labels_local_velocity_t=365~471.csv"
vps = 10
#You can change file name above.

#cell_movement = center_of_labels
cell_movement = np.zeros((center_of_labels.shape[0],center_of_labels.shape[1]))

for t in range(center_of_labels.shape[0]-1):
    for label in range(center_of_labels.shape[1]):
        if np.isnan(center_of_labels[0,label]).any():
            continue

        if np.isnan(center_of_labels[t+1,label]).any():
            center_of_labels[t+1,label] = center_of_labels[t,label]
        
        cell_position_t1 = center_of_labels[t,label]
        cell_position_t2 = center_of_labels[t+1,label]
        dist = distance.euclidean(cell_position_t1, cell_position_t2)
        cell_movement[t+1, label] = dist*0.4095*vps #umスケールに変換


csv_filename = os.path.join(tracked_labels_path, csv_filename)
timings = np.arange(1, cell_movement.shape[0]+1).reshape(-1, 1)
movement_with_timings = np.concatenate((timings, cell_movement), axis=1)
headers = ['timing'] + ['cell' + str(i)+'[µm]' for i in range(1, cell_movement.shape[1]+1)] 
np.savetxt(csv_filename, movement_with_timings, delimiter=',', fmt='%.3f', header=','.join(headers), comments='')
print(f"The cell movements have been saved in {csv_filename}")
    

## クマムシ動画用

In [6]:
from scipy.spatial import distance

csv_filename = r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=566~665\tardigrade_t=566~665_L4leg_labels_distance_1-2.csv"

#You can change file name above.

#cell_movement = center_of_labels
#cell_movement = np.zeros((center_of_labels.shape[0],center_of_labels.shape[1]-1))
cell_movement =np.zeros((center_of_labels.shape[0],4))
print(center_of_labels.shape)
label2_1_vector=np.zeros([center_of_labels.shape[0],3])
label2_1_vector[:,0]=center_of_labels[:,1,0] - center_of_labels[:,0,0]
label2_1_vector[:,1]=center_of_labels[:,1,1] - center_of_labels[:,0,1]
label2_1_vector[:,2]=center_of_labels[:,1,2] - center_of_labels[:,0,2]

print(label2_1_vector[1])
for t in range(center_of_labels.shape[0]):

    cell_position_t1 = center_of_labels[t,0]
    cell_position_t2 = center_of_labels[t,1]
    print(cell_position_t2)
    print(cell_position_t1)
    zx_center_of_labels_t1 = np.stack([cell_position_t1[0],cell_position_t1[2]],0)
    zx_center_of_labels_t2 = np.stack([cell_position_t2[0],cell_position_t2[2]],0)

    three_d_dist = distance.euclidean(cell_position_t1, cell_position_t2)
    xy_d_dist = distance.euclidean(cell_position_t1[1:2], cell_position_t2[1:2])
    xz_d_dist = distance.euclidean(zx_center_of_labels_t1, zx_center_of_labels_t2)
    yz_d_dist = distance.euclidean(cell_position_t1[0:1], cell_position_t2[0:1])

    cell_movement[t, 0] = three_d_dist*0.4095
    cell_movement[t, 1] = xy_d_dist*0.4095
    cell_movement[t, 2] = xz_d_dist*0.4095
    cell_movement[t, 3] = yz_d_dist*0.4095



csv_filename = os.path.join(tracked_labels_path, csv_filename)
timings = np.arange(1, cell_movement.shape[0]+1).reshape(-1, 1)
movement_with_timings = np.concatenate((timings, cell_movement), axis=1)
headers = ['timing'] + ['cell' + str(i) for i in range(1, cell_movement.shape[1])]
np.savetxt(csv_filename, movement_with_timings, delimiter=',', fmt='%.3f', header=','.join(headers), comments='')
print(f"The cell movements have been saved in {csv_filename}")

(100, 2, 3)
[ -9.18615353  10.19951569 -30.29931659]
[ 42.77572361 502.20495484 116.34097928]
[ 53.03006758 493.98987214 150.75467813]
[ 46.79293112 492.18233544 116.27723692]
[ 55.97908465 481.98281974 146.57655352]
[ 57.24771166 484.31524505 113.68544404]
[ 66.85951536 473.87306112 140.73314514]
[ 61.66413344 477.30650663 115.46189762]
[ 67.98785962 468.85172957 140.68535167]
[ 57.85221544 476.24728029 128.45095491]
[ 64.8814998  466.82063498 152.86749983]
[ 56.7223092  479.151254   134.42349786]
[ 63.74613236 467.85929471 157.92487482]
[ 51.29184418 481.35989975 144.29249149]
[ 57.8427364  472.81328475 168.70746758]
[ 46.17927293 481.30662515 147.40435855]
[ 57.54093055 471.67815091 169.94169234]
[ 34.9081896  489.06773607 148.60260495]
[ 45.64832641 477.91609606 175.7093843 ]
[ 26.43133926 489.18695321 141.65840703]
[ 36.97916807 481.77526446 178.01944848]
[ 18.78890998 489.14551094 143.25045469]
[ 31.14950933 482.92366445 179.87683936]
[ 18.05533988 494.35017343 147.28305983]
[ 28

###   クマムシ3次元座標のプロット

## ホヤz座標

In [ ]:


csv_filename = r"P:\Ascidiacea\ana\ascidiacea_MN2_zposition_reconst.csv"

#You can change file name above.

#cell_movement = center_of_labels
cell_movement = np.zeros((center_of_labels.shape[0],center_of_labels.shape[1]-1))

print(center_of_labels.shape)

for t in range(center_of_labels.shape[0]):
    #for label in range(center_of_labels.shape[1]):

        # if np.isnan(center_of_labels[0,label]).any():
        #     continue

        # if np.isnan(center_of_labels[t+1,label]).any():
        #     center_of_labels[t+1,label] = center_of_labels[t,label]
    print(center_of_labels.shape)

    # cell_position_t1 = center_of_labels[t,0]
    # cell_position_t2 = center_of_labels[t,1]
    # dist = distance.euclidean(cell_position_t1, cell_position_t2)
    cell_movement[t, 0] = center_of_labels[t,1,0]*0.4095


#csv_filename = os.path.join(tracked_labels_path, csv_filename)
timings = np.arange(1, cell_movement.shape[0]+1).reshape(-1, 1)
movement_with_timings = np.concatenate((timings, cell_movement), axis=1)
headers = ['timing,position(um)'] + ['cell' + str(i) for i in range(1, cell_movement.shape[1])]
np.savetxt(csv_filename, movement_with_timings, delimiter=',', fmt='%.3f', header=','.join(headers), comments='')
print(f"The cell movements have been saved in {csv_filename}")